In [10]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import datetime

In [11]:
# importing data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64


### Model Utility Function

In [12]:
def build_model(kernel_size: int, model_name: str) -> keras.Sequential:
    """
    Function that compiles and returns model with varrious kernel sizes and names
    """
    model = keras.Sequential([
      keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=kernel_size, 
                      strides=2, activation='relu', name='Conv1'),
      keras.layers.Flatten(),
      keras.layers.Dense(10, name='Dense')
    ])
    model._name= model_name
    model.summary()
    
    testing = False
    
    model.compile(optimizer='adam', 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy()])
    return model

In [13]:
model_a = build_model(15, "model_one")

Model: "model_one"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 7, 7, 8)           1808      
                                                                 
 flatten (Flatten)           (None, 392)               0         
                                                                 
 Dense (Dense)               (None, 10)                3930      
                                                                 
Total params: 5,738
Trainable params: 5,738
Non-trainable params: 0
_________________________________________________________________


In [14]:
model_b = build_model(3, "model_two")

Model: "model_two"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten_1 (Flatten)         (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________


In [15]:
# model training and logging metrics for tensorboard
epochs = 20
log_dir = "logs/"+str(model_a._name)+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_a.fit(train_images, 
          train_labels, 
          epochs = epochs,
          validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback])

Epoch 1/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.6029 - sparse_categorical_accuracy: 0.7869 - val_loss: 0.5255 - val_sparse_categorical_accuracy: 0.8147
Epoch 2/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4499 - sparse_categorical_accuracy: 0.8419 - val_loss: 0.4792 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4140 - sparse_categorical_accuracy: 0.8540 - val_loss: 0.4271 - val_sparse_categorical_accuracy: 0.8495
Epoch 4/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3927 - sparse_categorical_accuracy: 0.8616 - val_loss: 0.4216 - val_sparse_categorical_accuracy: 0.8534
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3774 - sparse_categorical_accuracy: 0.8652 - val_loss: 0.4019 - val_sparse_categorical_accuracy: 0.8591
Epoch 6/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3650 - spars

In [16]:
# model training and logging metrics for tensorboard
log_dir = "logs/"+str(model_b._name)+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_b.fit(train_images, 
          train_labels, 
          epochs = epochs,
          validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback])

Epoch 1/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5272 - sparse_categorical_accuracy: 0.8178 - val_loss: 0.4330 - val_sparse_categorical_accuracy: 0.8458
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3818 - sparse_categorical_accuracy: 0.8658 - val_loss: 0.4157 - val_sparse_categorical_accuracy: 0.8470
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3490 - sparse_categorical_accuracy: 0.8761 - val_loss: 0.3626 - val_sparse_categorical_accuracy: 0.8721
Epoch 4/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3291 - sparse_categorical_accuracy: 0.8826 - val_loss: 0.3561 - val_sparse_categorical_accuracy: 0.8745
Epoch 5/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3164 - sparse_categorical_accuracy: 0.8859 - val_loss: 0.3535 - val_sparse_categorical_accuracy: 0.8692
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3066 - spars

In [17]:
%load_ext tensorboard

In [18]:
%tensorboard --logdir logs/ --host 0.0.0.0

Reusing TensorBoard on port 6006 (pid 780), started 1:28:55 ago. (Use '!kill 780' to kill it.)

In [19]:
# saving the model to the docker volume
tf.keras.models.save_model(
    model_b,
    './model_output/1',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

INFO:tensorflow:Assets written to: ./model_output/1/assets


INFO:tensorflow:Assets written to: ./model_output/1/assets
